In [ ]:
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.animation import FuncAnimation, PillowWriter
from pot_analysis_helper_functions import *

# Example functions

In [ ]:
four_well_circuit_params = {
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0,   "phi_2x": 0, "phi_1xdc": 0, "phi_2xdc": 0, "M_12": 0
}

critical_point_circuit_params = {
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.61,   "phi_2x": 0.1, "phi_1xdc": 0, "phi_2xdc": 1.79, "M_12": 0.06
}

circuit_params = critical_point_circuit_params

In [ ]:
critical_dict = find_all_critical_points_for_all_potential(circuit_params, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
critical_points = list(critical_dict.values())[0]
critical_potential = [pot_function(circuit_params)([x, y]) for x, y in critical_points]
critical_points_color = ["green", "red", "yellow", "orange", "white", "white", "white", "white"]
# critical_points_color[2] = "pink"

solution_dict = find_all_minimum_points_for_all_potential(circuit_params, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2)])
solution_points = list(solution_dict.values())[0]
solution_points_color = ["green", "red", "yellow", "orange"]

# cutline_funciton

In [ ]:

def easy_plot_function(circuit_params, cutline_data_array):
    """
    This function is exactly the same as the following example: 
        2024/10/1 Example: plot cutlines on contour plot and show the potential of the cutline
    """
    fig, ax = plt.subplots(1,2, figsize = [8, 4])
    ax[0].set_xlim(-3, 3)
    ax[0].set_ylim(-3, 3)
    ax[0].set_aspect(1)

    # get the XYU
    resolution = 50
    X, Y, U = get_XYU(circuit_params, domain=[-4, 4], resolution=50)


    # plot contour_plot
    contour_plot = ax[0].contourf(X, Y, U, 40, vmin = 1, vmax = 6.5)


    # plot the critical points
    critical_dict = find_all_critical_points_for_all_potential(circuit_params, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]
    critical_potential = [pot_function(circuit_params)([x, y]) for x, y in critical_points]
    critical_points_color = ["green", "red", "yellow", "orange", "white", "white", "white", "white"]

    for (x, y), color in zip(critical_points, critical_points_color):
        ax[0].plot(x, y, marker = "d", markersize = 8, color = color)
        print(f"{color} = ({x:.3g}, {y:.3g})")
    # fig.colorbar(ScalarMappable(norm = contour_plot_2.norm, cmap=contour_plot_2.cmap))


    # plot cutlines
    ax_setting = {"ax": ax[1], "h_axis_label": r"$\varphi_2$", "v_axis_label": r"U", "X": X, "Y": Y, "U": U}

    for cutline_setting in cutline_data_array:
        # plot cutline on the right graph
        plotCutline(cutline_setting, ax_setting)

        # plot cutline on the contour graph
        if cutline_setting['direction'] == "v":
            ymin, ymax = ax[0].set_ylim()
            ax[0].vlines(cutline_setting['value'], ymin, ymax, color = cutline_setting['color'])
        else:
            xmin, xmax = ax[0].set_ylim()
            ax[0].hlines(cutline_setting['value'], xmin, xmax, color = cutline_setting['color'])

    # plot the cutline on the contour graph


In [ ]:

cutline_data_array = [
        {"value": 2.3, "direction": "v", "color": "red"},
        {"value": -1.75, "direction": "v", "color": "green"}
    ]

circuit_params = {
    "beta": 2.1, "d_beta": 0,
    "phi_1x": 0.61,   "phi_2x": 0.1, "phi_1xdc": 0, "phi_2xdc": 1.79, "M_12": 0.06
}
easy_plot_function(circuit_params = circuit_params, cutline_data_array = cutline_data_array)

In [ ]:
delta_U00 = critical_potential[1] - critical_potential[0]
delta_U11 = critical_potential[6] - critical_potential[3]
delta_U10 = critical_potential[6] - critical_potential[2]
delta_U01 = critical_potential[5] - critical_potential[4]

In [ ]:
print(delta_U11, delta_U10, delta_U01)

# sweeping parameter and show critical point

In [ ]:
def animate_sim_flux_qubit_with_cutline_and_projection(sweep_setting, param_sweeping_result=None, interval = 200, showPoint=None, **pot_kwargs):
    """
    This is very similar to the function animate_sim_flux_qubit_with_cutline, but it can project particles onto the cutlines
    """
    sweep_parameter, sweep_range = sweep_setting["parameter"], sweep_setting["range"]
    
    opacity = [0] * 8
    
    if showPoint != "all":
        for _i in showPoint:
            opacity[_i] = 1
    else:
        opacity = [1] * 8
    
    
    fig, ax = plt.subplots(1, 2, figsize=[10, 5])
    ax_flatten = ax.flatten()
    X, Y, U = param_sweeping_result[0]['contour_data'].values()
    ax[0].contourf(X, Y, U, 20)
    ax[0].set_title(f"{sweep_parameter} = {sweep_range[0]}")
    param_sweep_coord = param_sweeping_result[0]['coord']
    
    critical_points_color = ["green", "red", "yellow", "orange", "white", "white", "white", "white"]

    markers_array = []
    for _i, [(x, y), color] in enumerate(zip(param_sweep_coord, critical_points_color)):
        marker = ax[0].plot(x, y, marker = "d", markersize = 8, color = color, alpha = opacity[_i])
        markers_array.append(marker[0])
  
    def animate(i):
        X, Y, U = param_sweeping_result[i]['contour_data'].values()
        ax[0].contourf(X, Y, U, 20)
        ax[0].set_title(f"{sweep_parameter} = {sweep_range[i]:.3g}")
        param_sweep_coord = param_sweeping_result[i]['coord']
        for marker, (x,y) in zip(markers_array, param_sweep_coord):
            marker.set_xdata([x])
            marker.set_ydata([y])
            
        
    ani = FuncAnimation(fig, animate, interval=interval, frames=len(sweep_range), blit=False)
    return ani, fig, ax

In [ ]:
circuit_params

In [ ]:
sweep_setting = {"parameter": "beta", "range": np.linspace(2, 3, 10)}
param_sweeping_result = barrier_height_searching(critical_point_circuit_params, sweep_setting=sweep_setting)
param_sweep_coord = np.array([item['coord'] for item in param_sweeping_result])

In [ ]:
plt.scatter(sweep_setting['range'], param_sweep_coord[:, 0, 0])

In [ ]:
sweep_setting

In [ ]:
from IPython.display import HTML
from IPython import display

ani, _, _ = animate_sim_flux_qubit_with_cutline_and_projection(sweep_setting, param_sweeping_result, interval=500, showPoint="all")



video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
